In [ ]:
# A script to input and combine data for models
import pandas as pd
import tensorflow as tf
from pandas import DataFrame
import numpy as np
import math
import heapq
from tqdm import tqdm
import random
import ast
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#!pip install --upgrade tensorflow
#!pip install --upgrade pandas
#!pip install --upgrade numpy
#tf.debugging.set_log_device_placement(True)

In [ ]:
df = pd.read_csv('users-feeds.csv')
df = shrink_users_df(df, 'user')
df.loc[:, 'is_following_feed'] = 1

df = add_negative_samples(df,'feed_id','user','is_following_feed')

classes = pd.read_csv('feed_classifer_counts.csv')
print(classes.head())
print(list(classes['feed_classifier_counts'])[2])
classes['feed_classifier_counts'] = classes['feed_classifier_counts'].apply(lambda x: ast.literal_eval(x))

classes['feed_classifier_counts'] = classes['feed_classifier_counts'].apply(lambda x: gets_tags(x))

In [ ]:
df.head()



In [ ]:
classes.head()

In [ ]:
classes = classes[classes['feed_classifier_counts'].map(lambda x: x != [])]
classes.head()

In [ ]:

def match_feeds(classifier_df, users_df):
    """
    Takes a 2 dataframes, first one with `feed` & `feed_classifier_counts` columns
    Second other one with `feed_id` column
    
    Adds `feed_classifier` as a column to second df
    """
    
    # Merge dataframes    
    joined_df = users_df.merge(classifier_df[['feed_classifier_counts', 'feed']], how = 'left',
                    left_on = 'feed_id', right_on = 'feed').drop(columns= ['feed'])
    
    
    return joined_df

df_new = match_feeds(classes,df)
print(df_new.sample(20))

In [209]:
#reduce dups
def cleanse_double_tags(x):
    final_list = []
    for string in x:
        if string[0] == '"' and string[-1] == '"':
            final_list.append(string[1:-1])
        elif string[0] == "'" and string[-1] == "'":
            final_list.append(string[1:-1])
        else:
            final_list.append(string)
    return final_list

In [210]:
#reduce dups
def cleanse_hashtags(x):
    final_list = []
    for string in x:
        if string[0] == '#' and string[-1] != '#':
            final_list.append(string[1:])
        else: 
            final_list.append(string)
    return final_list

In [211]:
#df_new.loc[df_new.isnull()] = df_new.loc[df_new.isnull()].apply(lambda x: [])
df_new['feed_classifier_counts'] = df_new['feed_classifier_counts'].apply(lambda x: x if isinstance(x, list) else [])
df_new['feed_classifier_counts'] = df_new['feed_classifier_counts'].apply(lambda x: cleanse_double_tags(x))
df_new['feed_classifier_counts'] = df_new['feed_classifier_counts'].apply(lambda x: cleanse_hashtags(x))
df_new.sample(30)

,user,feed_id,is_following_feed,feed_classifier_counts
3440589,4320,428371,0,[]
6615474,208581,392071,0,[]
4799005,61931,1125293,1,[]
3548720,167211,6206,1,[]
5281491,79523,1848670,0,[]
6875192,545110,4442404,0,[]
1407914,187770,4676380,0,[]
3246024,669,3997446,0,[]
4346237,59273,3334971,0,[]
6000666,106209,5795266,0,[]


In [212]:
def gets_tags(x):
  tags = x.get('tag')
  final_list = []
  if tags == None:
    return []
  for item in tags:
    final_list.append(item.get('tag'))
  return final_list


def try_conversion(x):
    try:
        return ast.literal_eval(x)
    except Exception as e:
        return {}
def shrink_users_df(df,user_id):
    userIds = np.random.choice(df[user_id].unique(),
                                    size=int(len(df[user_id].unique())*0.10),
                                    replace=False)
    return df.loc[df[user_id].isin(userIds)]
def add_negative_samples(df, item_tag, user_tag,label_tag):

    updated_df = pd.DataFrame(columns=[user_tag,item_tag,label_tag])
    all_feeds = df[item_tag].unique()
    users, items, labels = [], [], []

    user_item_set = set(zip(df[user_tag], df[item_tag]))
    num_negatives = 4

    for (u, i) in user_item_set:
        users.append(u)
        items.append(i)
        labels.append(1) # items that the user has interacted with are positive
        for _ in range(num_negatives):
            # randomly select an item
            negative_item = np.random.choice(all_feeds)
            # check that the user has not interacted with this item
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_feeds)
            users.append(u)
            items.append(negative_item)
            labels.append(0) # items not interacted with are negative
    updated_df[user_tag] = users
    updated_df[item_tag] = items
    updated_df[label_tag] = labels
    del df
    return updated_df

def mask_first(x):
    """
    Return a list of 0 for the first item and 1 for all others
    """
    result = np.ones_like(x)
    result[0] = 0

    return result

# needs to add validate in the future
def train_test_split(full_df):
    df_test = df.copy(deep=True)
    df_test = df_test.groupby(['user']).first()

    df_test['user'] = df_test.index
    df_test = df_test[['user', 'feed_id','is_following_feed']]
    df_test = df_test.rename_axis(None, axis=1)

    df_train = df.copy(deep=True)
    mask = df.groupby(['user'])['user'].transform(mask_first).astype(bool)

    df_train = df.loc[mask]
    return df_train, df_test

In [213]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()




In [224]:
encode = mlb.fit(df_new['feed_classifier_counts'].tolist())


MultiLabelBinarizer()

In [215]:
len(list(encode.classes_))

126605

In [216]:
list(encode.classes_)[6:]

['" again?',
 '">community<',
 '"creative" editing',
 '"final fantasy vii remake" (video game)',
 '"hardcore" gaming',
 '"helpful" toddlers',
 '"the less they know',
 '"új" iráni vadászgép',
 '#',
 '$$',
 '$1 and free',
 '$10 and beyond',
 '$100 startup',
 '$100-$250',
 '$149',
 '$199',
 '$25-$50',
 '$4.99',
 '$5 and under',
 '$99 xbox 360 kinect bundle',
 '$apple watch series 2',
 '$cl_f',
 '$dia',
 '$dx_f',
 '$eem',
 '$fxe',
 '$gc_f',
 '$gld',
 '$indu',
 '$iwm',
 '$mtum',
 '$ng_f',
 '$qqq',
 '$si_f',
 '$slv',
 '$spx',
 '$spy',
 '$uicideboy$',
 '$uicideboy$ and germ',
 '$uicideboys',
 '$uup',
 '$vlue',
 '$xle',
 '$z.99',
 '& cemeteries',
 '& eco',
 '& food',
 '& health',
 '& paper',
 '& productivity',
 '& storage',
 '&apos;n sync',
 "'77",
 "'80's",
 "'90s",
 "'90s fashion",
 "'90s nostalgia",
 "'cool websites and tools' series",
 "'n sync",
 "'ndrangheta",
 "'ndrangheta al nord",
 "'nude' airport body scanners",
 "'ship shape",
 "'tocks",
 "'tocktober 2014",
 "'tocktober 2015",
 "'va

In [220]:
"'90s" in list(encode.classes_)

True

In [221]:
"90s" in list(encode.classes_)

True

In [222]:
list(encode.classes_).index("'90s")

60

In [223]:
list(encode.classes_).index("90s")

1607